## 챌린지
---
(EN)
- Make a chain that takes the name of a movie and replies with information about the movie such as the director, the main cast, the budget, the box office revenue, the genre of the movie and a small synopsis.
- The LLM should always reply using the same format, to accomplish this you must give examples to the LLM of your desired output.
- To provide examples use FewShotPromptTemplate or FewShotChatMessagePromptTemplate.

(KR)
- 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
- LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
- 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

In [7]:
import os
from dotenv import load_dotenv

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
llm = ChatOpenAI(
    api_key=openai_api_key,
    temperature=0.5,
)

examples = [
    {
        "movie": "Inception",
        "response": """
        Director: Christopher Nolan
        Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page
        Budget: $160 million
        Box Office: $829.9 million
        Genre: Sci-Fi, Action
        Synopsis: A skilled thief is given a chance at redemption if he can successfully plant an idea into a target's subconscious.
        """
    },
    {
        "movie": "The Godfather",
        "response": """
        Director: Francis Ford Coppola
        Main Cast: Marlon Brando, Al Pacino, James Caan
        Budget: $6 million
        Box Office: $250 million
        Genre: Crime, Drama
        Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
        """
        
    }
]

In [12]:
# 형식 지정 
prompt = ChatPromptTemplate.from_messages([
    ("user", "Tell me about the infomations of {movie}"),
    ("ai", "{response}")
])

# 형식에 맞게 대화형으로 답변 
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)

# 초기 설정, 예제와 함께 새로운 질문 
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie information expert. Given a movie name, provide details"),
    example_prompt,
    ("user", "Tell me about the infomations of {movie}")
])

chain = final_prompt | llm

In [14]:
movie_name = "Interstellar"

answer = chain.invoke({"movie": movie_name}).content
print(answer)


        Director: Christopher Nolan
        Main Cast: Matthew McConaughey, Anne Hathaway, Jessica Chastain
        Budget: $165 million
        Box Office: $677.5 million
        Genre: Sci-Fi, Drama
        Synopsis: A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.
        


## 정답코드

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
examples = [
    {
        "movie": "Iron Man",
        "answer": """
        Information of Iron Man
        Director: Jon Favreau
        Main Cast: Robert Downey Jr., Gwyneth Paltrow, Jeff Bridges, Terrence Howard
        Budget: $140 million
        Box Office Revenue: $585.8 million
        Genre: Action, Science Fiction, Superhero
        Synopsis: After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.
        """,
    },
    {
        "movie": "Thor",
        "answer": """
        Information of Thor
        Director: Kenneth Branagh
        Main Cast: Chris Hemsworth, Natalie Portman, Tom Hiddleston, Anthony Hopkins
        Budget: $150 million
        Box Office Revenue: $449.3 million
        Genre: Action, Fantasy, Superhero
        Synopsis: The powerful but arrogant god Thor is cast out of Asgard to live amongst humans in Midgard (Earth), where he soon becomes one of their finest defenders.
        """,
    },
    {
        "movie": "Spider-Man",
        "answer": """
        Information of Spider-Man
        Director: Sam Raimi
        Main Cast: Tobey Maguire, Kirsten Dunst, Willem Dafoe, James Franco
        Budget: $139 million
        Box Office Revenue: $825 million
        Genre: Action, Adventure, Superhero
        Synopsis: When bitten by a genetically modified spider, a nerdy high school student gains spider-like abilities that he must use to fight evil as a superhero after tragedy befalls his family.
        """,
    },
]

In [4]:
# FewShotPromptTemplate를 활용

example_prompt = PromptTemplate.from_template(
    "Human: Tell me about {movie} movie.\nAI: {answer}"
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: Tell me about {movie} movie",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "The Avengers"})

AI: 
        Information of The Avengers
        Director: Joss Whedon
        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner, Tom Hiddleston, Samuel L. Jackson
        Budget: $220 million
        Box Office Revenue: $1.519 billion
        Genre: Action, Adventure, Superhero
        Synopsis: Earth's mightiest heroes, including Iron Man, Captain America, Thor, and the Hulk, must come together to stop the villainous Loki and his alien army from conquering Earth.

AIMessage(content="AI: \n        Information of The Avengers\n        Director: Joss Whedon\n        Main Cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, Jeremy Renner, Tom Hiddleston, Samuel L. Jackson\n        Budget: $220 million\n        Box Office Revenue: $1.519 billion\n        Genre: Action, Adventure, Superhero\n        Synopsis: Earth's mightiest heroes, including Iron Man, Captain America, Thor, and the Hulk, must come together to stop the villainous Loki and his alien army from conquering Earth.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-4112799d-8516-410e-afab-59c8a3a252f8-0')